In [65]:
import time
import pandas as pd

# Definição de constantes

In [66]:
limite_valor_saque = 500
limite_n_saque = 3

In [67]:
valor_inicial = 2000

df = pd.DataFrame(columns=['Valor_Conta', 'Data_Hora', 'Tipo_Operacao'])
init_data = pd.DataFrame({'Valor_Conta': [valor_inicial],
                            'Data_Hora': [time.strftime("%d/%m/%Y %H:%M")],
                            'Tipo_Operacao': ['I']})
df = pd.concat([df, init_data], ignore_index=True)
df

,Valor_Conta,Data_Hora,Tipo_Operacao
0,2000,20/02/2024 08:27,I


# Definição das funções operacionais

In [68]:
def operacao_deposito(valor_deposito, df):
    # Check if the input is numeric
    try:
        valor_deposito = float(valor_deposito)
    except ValueError:
        print("""
              O valor informado não é um número válido, favor tentar novamente
              ----------------------------------------------------------------------

              Importante:
              O presente sistema utiliza '.' como identificador decimal.
              Para realizar o depósito, insira apenas um único valor numérico.  
              Não utilize letras ou símbolos especiais.
        """)
        return df

    if valor_deposito > 0:
        current_datetime = time.strftime("%d/%m/%Y %H:%M")

        new_data = pd.DataFrame({'Valor_Conta': [valor_deposito],
                                 'Data_Hora': [current_datetime],
                                 'Tipo_Operacao': ['D']})
        df = pd.concat([df, new_data], ignore_index=True)
        print(f'Depósito realizado em: {current_datetime}')
        print(f'Depósito no valor de R$ {valor_deposito:.2f} realizado com sucesso.')
    else:
        print('Não é possível realizar um depósito nulo ou com valor negativo.')

    return df

In [69]:
def operacao_saque(valor_saque, df_transacoes):
    # Limite de 3 saques diários com limite de no máximo de 500/saque
    # Caso não tenha saldo em conta, o sistema deve exibir uma mensagem
    # Todos os saque devem ser armazenados em uma variável e exibidos na operação de Extrato

    # Verificar se há saldo suficiente
    saldo_conta = df_transacoes['Valor_Conta'].sum()
    if saldo_conta < valor_saque:
        print("Saldo insuficiente para realizar o saque.")
        return df_transacoes

    # Verificar se o valor do saque é válido
    if valor_saque <= 0 or valor_saque > limite_valor_saque:
        print(f"Valor de saque inválido. O valor do saque não pode ser negativo ou maior do que R$ {limite_valor_saque:.2f}.")
        return df_transacoes

    # Limitar a 3 saques diários
    data_hoje = time.strftime("%d/%m/%Y")
    saques_hoje = df_transacoes[(df_transacoes['Data_Hora'].str[:10] == data_hoje) & (df_transacoes['Tipo_Operacao'] == 'S')].shape[0]
    if saques_hoje >= limite_n_saque:
        print(f"Limite diário de saques atingido. Você já realizou {limite_n_saque:.0f} saques hoje.")
        return df_transacoes

    # Atualizar o DataFrame de transações
    saldo_conta -= valor_saque
    current_datetime = time.strftime("%d/%m/%Y %H:%M")

    new_data = pd.DataFrame({'Valor_Conta': [-valor_saque],'Data_Hora': [current_datetime],'Tipo_Operacao': ['S']})
    df_transacoes = pd.concat([df_transacoes, new_data], ignore_index=True)
    
    print(f'Saque realizado em: {current_datetime}')
    print(f'Saque no valor de R$ {valor_saque:.2f} realizado com sucesso.')

    return df_transacoes


In [70]:
def operacao_extrato(df_transacoes):
    # Essa operação deve listar todos os depósitos e saques realizados na conta
    # No fim da listagem deve ser exibido o atual da conta
    # Os valores devem ser exibidos utilizando o formato R$ xxx.xx

    # Listar todos os depósitos e saques realizados na conta
    print("Extrato de transações:")
    for index, row in df_transacoes.iterrows():
        valor_transacao = row['Valor_Conta']
        if valor_transacao > 0:
            tipo_transacao = "Depósito"
        else:
            tipo_transacao = "Saque"
        print(f"{tipo_transacao}: R$ {abs(valor_transacao):.2f} - Data/Hora: {row['Data_Hora']}")

    # Exibir o saldo atual da conta
    saldo_conta = df_transacoes['Valor_Conta'].sum()
    print(f"\nSaldo atual da conta: R$ {saldo_conta:.2f}")


# Implementação

## Depósito

In [71]:
df = operacao_deposito(100.50, df)
df

Depósito realizado em: 20/02/2024 08:27
Depósito no valor de R$ 100.50 realizado com sucesso.


,Valor_Conta,Data_Hora,Tipo_Operacao
0,2000,20/02/2024 08:27,I
1,100.5,20/02/2024 08:27,D


## Saque

### Testar um saque válido

In [72]:
df = operacao_saque(200, df)
print("Saldo atual da conta:", df['Valor_Conta'].sum())

Saque realizado em: 20/02/2024 08:27
Saque no valor de R$ 200.00 realizado com sucesso.
Saldo atual da conta: 1900.5


### Testar um saque inválido (valor maior que o saldo)

In [73]:
df = operacao_saque(1500, df)

Valor de saque inválido. O valor do saque não pode ser negativo ou maior do que R$ 500.00.


### Testar um saque inválido (valor negativo)

In [74]:
df = operacao_saque(-50, df)

Valor de saque inválido. O valor do saque não pode ser negativo ou maior do que R$ 500.00.


### Testar um saque inválido (limite diário de saques atingido)

In [75]:
df = operacao_saque(100, df)
df = operacao_saque(100, df)
df = operacao_saque(100, df)
df = operacao_saque(100, df)

Saque realizado em: 20/02/2024 08:27
Saque no valor de R$ 100.00 realizado com sucesso.
Saque realizado em: 20/02/2024 08:27
Saque no valor de R$ 100.00 realizado com sucesso.
Limite diário de saques atingido. Você já realizou 3 saques hoje.
Limite diário de saques atingido. Você já realizou 3 saques hoje.


## Extrato

In [76]:
operacao_extrato(df)

Extrato de transações:
Depósito: R$ 2000.00 - Data/Hora: 20/02/2024 08:27
Depósito: R$ 100.50 - Data/Hora: 20/02/2024 08:27
Saque: R$ 200.00 - Data/Hora: 20/02/2024 08:27
Saque: R$ 100.00 - Data/Hora: 20/02/2024 08:27
Saque: R$ 100.00 - Data/Hora: 20/02/2024 08:27

Saldo atual da conta: R$ 1700.50
